In [4]:
import numpy as np
from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.layers import Input, Conv2D, Dropout,  Dense, GlobalAveragePooling2D, Concatenate, AveragePooling2D
from keras.layers import Activation, BatchNormalization, add, Reshape, ReLU, DepthwiseConv2D, MaxPooling2D, Lambda
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.optimizers import SGD

In [2]:
def _group_conv(x, filters, kernel, stride, groups): 
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    in_channels = K.int_shape(x)[channel_axis]
    # number of input channels per group
    nb_ig = in_channels // groups
    # number of output channels per group
    nb_og = filters // groups
    gc_list = []
    # Determine whether the number of filters is divisible by the number of groups
    assert filters % groups == 0
    for i in range(groups):
        if channel_axis == -1:
            x_group = Lambda(lambda z: z[:, :, :, i * nb_ig: (i + 1) * nb_ig])(x)
        else:
            x_group = Lambda(lambda z: z[:, i * nb_ig: (i + 1) * nb_ig, :, :])(x)
        gc_list.append(Conv2D(filters=nb_og, kernel_size=kernel, strides=stride, 
                              padding='same', use_bias=False)(x_group))
    return Concatenate(axis=channel_axis)(gc_list)

def _channel_shuffle(x, groups):  
    if K.image_data_format() == 'channels_last':
        height, width, in_channels = K.int_shape(x)[1:]
        channels_per_group = in_channels // groups
        pre_shape = [-1, height, width, groups, channels_per_group]
        dim = (0, 1, 2, 4, 3)
        later_shape = [-1, height, width, in_channels]
    else:
        in_channels, height, width = K.int_shape(x)[1:]
        channels_per_group = in_channels // groups
        pre_shape = [-1, groups, channels_per_group, height, width]
        dim = (0, 2, 1, 3, 4)
        later_shape = [-1, in_channels, height, width]
    x = Lambda(lambda z: K.reshape(z, pre_shape))(x)
    x = Lambda(lambda z: K.permute_dimensions(z, dim))(x)  
    x = Lambda(lambda z: K.reshape(z, later_shape))(x)
    return x

def _shufflenet_unit(inputs, filters, kernel, stride, groups, stage, bottleneck_ratio=0.25): 
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    in_channels = K.int_shape(inputs)[channel_axis]
    bottleneck_channels = int(filters * bottleneck_ratio)
    if stage == 2:
        x = Conv2D(filters=bottleneck_channels, kernel_size=kernel, strides=1,
                   padding='same', use_bias=False)(inputs)
    else:
        x = _group_conv(inputs, bottleneck_channels, (1, 1), 1, groups)
    x = BatchNormalization(axis=channel_axis)(x)
    x = ReLU()(x)
    x = _channel_shuffle(x, groups)
    x = DepthwiseConv2D(kernel_size=kernel, strides=stride, depth_multiplier=1, 
                        padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    if stride == 2:
        x = _group_conv(x, filters - in_channels, (1, 1), 1, groups)
        x = BatchNormalization(axis=channel_axis)(x)
        avg = AveragePooling2D(pool_size=(3, 3), strides=2, padding='same')(inputs)
        x = Concatenate(axis=channel_axis)([x, avg])
    else:
        x = _group_conv(x, filters, (1, 1), 1, groups)
        x = BatchNormalization(axis=channel_axis)(x)
        x = add([x, inputs])
    return x
def _stage(x, filters, kernel, groups, repeat, stage): 
    x = _shufflenet_unit(x, filters, kernel, 2, groups, stage)
    for i in range(1, repeat):
        x = _shufflenet_unit(x, filters, kernel, 1, groups, stage)
    return x

In [3]:
def ShuffleNet(input_shape, classes): 
    inputs = Input(shape=input_shape)
    x = Conv2D(24, (3, 3), strides=2, padding='same', use_bias=True, activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    x = _stage(x, filters=384, kernel=(3, 3), groups=8, repeat=4, stage=2)
    x = _stage(x, filters=768, kernel=(3, 3), groups=8, repeat=8, stage=3)
    x = _stage(x, filters=1536, kernel=(3, 3), groups=8, repeat=4, stage=4)
    x = GlobalAveragePooling2D()(x)    
    x = Dense(classes)(x)
    predicts = Activation('softmax')(x)
    model = Model(inputs, predicts)
    return model
model = ShuffleNet((224, 224, 3), 1000) 
model.summary()

# 把模型保存为图片
from keras.utils import plot_model
plot_model(model,to_file='model_png/210ShffuleNet.png', show_layer_names=True, show_shapes=True)

W0808 17:25:27.109282 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 17:25:27.177519 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 17:25:27.191881 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 17:25:27.230511 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 24) 672         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 24)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 96)   20736       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
batch_norm

__________________________________________________________________________________________________
lambda_141 (Lambda)             (None, 14, 14, 96)   0           add_7[0][0]                      
__________________________________________________________________________________________________
lambda_142 (Lambda)             (None, 14, 14, 96)   0           add_7[0][0]                      
__________________________________________________________________________________________________
lambda_143 (Lambda)             (None, 14, 14, 96)   0           add_7[0][0]                      
__________________________________________________________________________________________________
lambda_144 (Lambda)             (None, 14, 14, 96)   0           add_7[0][0]                      
__________________________________________________________________________________________________
lambda_145 (Lambda)             (None, 14, 14, 96)   0           add_7[0][0]                      
__________

Total params: 3,473,800
Trainable params: 3,434,680
Non-trainable params: 39,120
__________________________________________________________________________________________________
